In [2]:
import json

from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

In [3]:
ds_path = '../../data/topic_extraction/keywords_gemma_superseg.json'
with open(ds_path, 'r') as f:
    data = json.load(f)['data']
data

[{'key_words': ['Benefits', 'Future', 'Planning', 'Future', 'Information'],
  'topic_id': 0,
  'dialogue_id': 0},
 {'key_words': ['Reduced benefits',
   'Age 60',
   'Retirement age',
   'Benefits',
   'Full benefits'],
  'topic_id': 1,
  'dialogue_id': 0},
 {'key_words': ['Social Security', 'Credits', 'Working', 'Benefits', 'Family'],
  'topic_id': 2,
  'dialogue_id': 0},
 {'key_words': ['Apply', 'Disability', 'Benefits', 'Deaf', 'Telephone'],
  'topic_id': 0,
  'dialogue_id': 1},
 {'key_words': ['Disability benefits',
   'Starter kit',
   'Application',
   'Documents',
   'Starter'],
  'topic_id': 1,
  'dialogue_id': 1},
 {'key_words': ['Social Security Card',
   'Citizenship',
   'American',
   'Documents',
   'Establish'],
  'topic_id': 0,
  'dialogue_id': 2},
 {'key_words': ['Citizen', 'Identity', 'US passport', 'Proof', 'Documents'],
  'topic_id': 1,
  'dialogue_id': 2},
 {'key_words': ['Passport',
   'Social Security Card',
   'Name',
   'Marriage',
   'Identity'],
  'topic_id':

In [19]:
sample = data[0]
sample['topics'] = [[word.lower() for word in sample['key_words']]]
text = 'I need help planning my Social Security benefits for my survivors.\nAre you currently planning for your future?\nYes, I am.\nyou should think about what your family would need if you die now. Social Security can help your family if you have earned enough credits through your work.'.split('.')
texts = []
for sent in text:
    cur_sent = [word.lower() for word in sent.split()]
    if len(cur_sent) != 0:
        texts.append(cur_sent)

td_metric = TopicDiversity(topk=5) # Initialize metric
topic_diversity_score = td_metric.score(sample)

coh_metric = Coherence(topk=5, texts=texts)
coherence_score = coh_metric.score(sample)

print(topic_diversity_score, coherence_score)

0.8 0.7833824999777799
